## Creating an MCP Server

### Sample Runs

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [5]:
from accounts import Account

In [9]:
account = Account.get("Ed")
account

Account(name='ed', balance=9822.646, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 59.118 each.], portfolio_value_time_series=[('2025-11-29 19:45:03', 10038.646)])

In [10]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9630.262, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 59.118, "timestamp": "2025-11-29 19:45:03", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-11-29 19:45:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-29 19:45:03", 10038.646], ["2025-11-29 19:45:30", 10188.262]], "total_portfolio_value": 10188.262, "total_profit_loss": 188.26200000000063}'

In [11]:
account.report()

'{"name": "ed", "balance": 9630.262, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 59.118, "timestamp": "2025-11-29 19:45:03", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-11-29 19:45:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-29 19:45:03", 10038.646], ["2025-11-29 19:45:30", 10188.262], ["2025-11-29 19:45:45", 9684.262]], "total_portfolio_value": 9684.262, "total_profit_loss": -315.7379999999994}'

In [12]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 59.118,
  'timestamp': '2025-11-29 19:45:03',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 64.128,
  'timestamp': '2025-11-29 19:45:30',
  'rationale': 'Because this bookstore website looks promising'}]

### Using MCP Server

In [13]:
# use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [14]:
mcp_tools

[Tool(name='get_balance', description='\n    Get the cash balance of the account for the name given as the input.\n\n    Args:\n        The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name'

In [15]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [16]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your account balance is $9,630.26. Your current holdings include 6 shares of Amazon (AMZN). Is there anything specific you would like to do or know about your account?

## Creating the MCP Client

In [17]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='\n    Get the cash balance of the account for the name given as the input.\n\n    Args:\n        The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': 

In [18]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your current account balance is $9,630.26. If you need any other information or want to make any transactions, just let me know!

In [19]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9630.262, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 59.118, "timestamp": "2025-11-29 19:45:03", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-11-29 19:45:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-29 19:45:03", 10038.646], ["2025-11-29 19:45:30", 10188.262], ["2025-11-29 19:45:45", 9684.262], ["2025-11-29 21:15:05", 10134.262]], "total_portfolio_value": 10134.262, "total_profit_loss": 134.26200000000063}


In [20]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9630.262, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 59.118, "timestamp": "2025-11-29 19:45:03", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-11-29 19:45:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-29 19:45:03", 10038.646], ["2025-11-29 19:45:30", 10188.262], ["2025-11-29 19:45:45", 9684.262], ["2025-11-29 21:15:05", 10134.262], ["2025-11-29 21:15:05", 10170.262]], "total_portfolio_value": 10170.262, "total_profit_loss": 170.26200000000063}'